<a href="https://colab.research.google.com/github/Valtern/machinelearningsmt5/blob/main/Midterm_MachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import google.colab.drive
google.colab.drive.mount('/content/drive')

Mounted at /content/drive


# Case Study 2 - CC General

- This case study relates to credit card user data.
- For students with attendance numbers 2, 5, 8, 11, etc.
- Follow these steps and analyze the results.

## Data Preprocessing (20 points)

1. Load data (5 points)
2. Perform data imputation on missing data. Attach your analysis explaining why you used the chosen imputation method. (10 points)
3. Drop irrelevant features (5 points)

## Clustering - 1 (30 points)

1. Create a cluster model using KMeans with the number of clusters from 1 to 10. (10 points)
2. Create an elbow diagram, determine the best number of clusters according to the elbow method, and provide your reasoning. (10 points)
3. Calculate the silhouette coefficient value for k=1 to k=10. Based on the silhouette value, what is the best number of clusters? Is it the same as the elbow method? (10 points)

NB: You are allowed to perform normalization/standardization before building the clustering model. Observe the clustering results.

## Clustering - 2 (20 points)

1. Create a cluster model using DBSCAN. How many clusters are produced? (10 points)
2. What are the silhouette coefficient and DBI values of the clusters produced by DBSCAN? (10 points)

## Feature Selection (30 points)
After knowing the performance of the clustering results on the KMeans and DBSCAN models based on the silhouette coefficient and DBI, perform a feature selection process to improve the silhouette coefficient and DBI values. You are allowed to use an iterative process or other methods.

1. Which features provide the best silhouette coefficient/DBI values? Include the feature selection process.
2. Why did you use that feature selection method?

## BONUS - CHALLENGE (30 points)
Build an ANNOY model using the data that has gone through the data preprocessing stage. Perform a query with the 5 nearest values. Does ANNOY query the best cluster you have formed?